In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Introduction to Distributed Computing
<!-- requirement: small_data/gutenberg -->
<!-- requirement: scripts/mapper.py -->
<!-- requirement: scripts/reducer.py -->
<!-- requirement: images/MapReduce_Work_Structure.png -->

## Big Data

Big data consists of large, growing files on the order of terabytes ($10^{12}$) and petabytes ($10^{15}$). Big data includes (among other things) transactional, sensor, and social data, which are typically unstructured and can exist in a variety of formats like text, audio, and visual files.  Such data sets present challenges in capturing, storing, searching, visualizing, and querying information. Distributed computing is often the solution to these big data sets.

Such large data sets require a qualitative change in how the data is analyzed.  When your data fit in memory, you don't have to worry (much) about the order of operations.  It will take ~100ns to access any random memory location.

When your data do not fit in memory, you will have to hit the disk to load the portion you need at any time.  Random access from the disk will take from ~100μs (for SSDs) to ~10ms (for spinning platters).  This is a factor of 1,000 to 100,000 slower than memory!

However, these numbers are for random access.  Any consumer hard drive can easily give a sequential read speed of 200MB/s.  This corresponds to a byte every 5ns, twenty times faster than reading random bytes from memory.  In order to be performant, algorithms for large data sets must be able to consider only a subset of data (ideally sequential) at a time.

## Distributed Computing

Once an algorithm has been written to work on only a subset of data at a time, it is relatively easily to divide the data amongst several machines and run the task in parallel.  This is the basic idea behind **distributed computing**.  Several computers will each compute part of an answer by themselves, and then those pieces will be combined into a final result.  Within a data center, network latencies (~1ms) are roughly equivalent to hard drive latencies, so distributing data to different nodes isn't too much worse than reading it off of a local hard drive.

Not all tasks can be easily completed in a distributed computing framework. Tasks need to be divisible such that many operations can take place independent from each other.  Problems that are obviously well-suited to this framework are called **embarrassingly parallel**, as they can be parallelized to many machines without reworking the algorithm.  Examples include 3D video rendering (each frame is independent) and growing random forests (each tree is independent).  However, algorithms that require many steps which each depend on the result of the previous do not parallelize as easily.  For instance, each step of a gradient descent algorithm needs to know the parameters from the previous step, so it is not embarrassingly parallel.  (It can be, and often is, parallelized, but more work must be done to harmonize the gradient calculations from the disparate calculations.)

While it is "relatively easy" to make an embarrassingly parallel algorithm work in a distributed manner, there is a lot of bookkeeping to be done in keeping track of data, distributing jobs, collecting results, and dealing with failures.  Fortunately, tools like Spark provide all of this functionality, letting us worry about the algorithm itself.

## MapReduce: A simple distributed-computing framework

Keeping state synchronized across multiple nodes is difficult.  Many distributed systems adopt a **functional programming** approach, with
- Pure functions (with no side effects)
- Immutable data

In December, 2004, Jeffery Dean and Sanjay Ghemawat of Google introduced [MapReduce](https://research.google.com/archive/mapreduce.html), a scheme for distributed computing based on two higher-order functions: **map** and **reduce**.  Both exist in Python:

**Map**: Map operations are trivially parallelizable.

In [ ]:
def square(x):
    return x * x

#The list command is to make a readable output
#in python3 map and filter create generators
list(map(square, range(10)))

**Reduce**: Reduce operations should be commutative [$a \star b = b \star a$] and associative [$(a \star b) \star c = a \star (b \star c)$], so they can be evaluated in arbitrary order

In [ ]:
from functools import reduce

def add(x, y):
    return x + y

reduce(add, map(square, range(10)))

These functions are often specified with `lambda`, but that is only an issue of syntax.

In [ ]:
reduce(lambda x, y: x + y, map(lambda x: x * x, range(10)))

**Example**: Find the sum of squares of 1 million numbers

On 1 machine:
1. $10^6$ steps to square each number
1. $10^6$ steps to add all the numbers
1. $\mathcal{O}(n)$

On 1 million machines:
1. Map: 1 step to square all of the numbers
1. Reduce: $\log_2 (10^6) \approx 20$ steps to add all the numbers
1. $\mathcal{O}(\log n)$

## Word Count: The "Hello World" of distributed computing

The "word count" problem is a good example of an embarassingly parallel problem, so we use it to demonstrate the thinking around distributed computing.  The problem is to get a count of how many times each word appears in some text.  We can imagine doing this in real life, with two different approaches.

**Serial approach:** Read through a book from cover to cover, keeping a tally.

**Parallel approach:** Give pages to a bunch of people, have them tally each page, then sum the tallies.

We will show how to execute this parallel approach in several frameworks.

### MapReduce with Unix pipes

We will write mapper and reducer programs.  We can use Unix pipes to connect the output of the mapper to the reducer.

The mapper program takes in text a line at a time and outputs a count of one for each word seen.  (Technically, this is an example of a *flatmap*, since each input gives multiple outputs, but these are flattened into a single output list.)  The output is in the form of key-value pairs, where the key is the word and the value the count.

#### `mapper.py`
```python
import sys

def read_input(file, encoding='utf-8'):
    for line in file:
        yield line.decode(encoding)

def write_line(line, encoding='utf-8'):
    sys.stdout.buffer.write((line + '\n').encode(encoding))

def main(separator='\t'):
    data = read_input(sys.stdin.buffer)
    for line in data:
        for word in line.split():
            write_line('%s%s%d' % (word, separator, 1))

if __name__ == "__main__":
    main()
```

The reducer combines groups with the same word to count up the sum.  Note that `itertools.groupby` only creates groups out of sequential inputs.  This is beneficial, since it means that we don't have to read all the input into memory.  It does require that the input to the reducer be sorted by key.

#### `reducer.py`
```python
from itertools import groupby
from operator import itemgetter
import sys

def read_input(file, encoding='utf-8'):
    for line in file:
        yield line.decode(encoding)

def write_line(line, encoding='utf-8'):
    sys.stdout.buffer.write((line + '\n').encode(encoding))

def main(separator='\t'):
    data = map(lambda line: line.split(separator, 1),
               read_input(sys.stdin.buffer))
    for current_word, group in groupby(data, itemgetter(0)):
        total_count = sum(int(count) for _, count in group)
        write_line("%s%s%d" % (current_word, separator, total_count))

if __name__ == "__main__":
    main()
```

One the command line, we can create a pipeline to process the data.  Note the use of `sort` prior to the reducer.
```bash
cat small_data/gutenberg/* \
    | python scripts/mapper.py \
    | sort -k1,1 \
    | python scripts/reducer.py \
    | grep "^[a-z]" \
    | less
```

We can run the map and reduce phases in parallel with the `parallel` command.
```bash
cat small_data/gutenberg/* \
    | parallel --pipe python scripts/mapper.py \
    | sort -k1,1 \
    | parallel --pipe python scripts/reducer.py \
    | grep "^[a-z]" \
    | less
```

(Note that the reducer doesn't quite work in parallel, since we haven't ensured that all copies of a key will go to same reducer.)

### Hadoop MapReduce

Hadoop is a distributed-computing framework, written in Java.  Its MapReduce example is nearly as simple as the above.

```java
package com.thedataincubator.hadoopexamples;

import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
import org.apache.hadoop.util.GenericOptionsParser;

public class WordCount {

  public static class WordTokenizerMapper
  extends Mapper<Object, Text, Text, IntWritable>
  {
    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(Object key,
                    Text value,
                    Context context)
    throws IOException, InterruptedException
    {
      //System.err.println(String.format("[map] key: (%s), value: (%s)", key, value));
      // break each sentence into words, using the punctuation characters shown
      StringTokenizer tokenizer = new StringTokenizer(value.toString(), " \t\n\r\f,.:;?![]'\"");
      while (tokenizer.hasMoreTokens())
      {
        // make the words lowercase so words like "an" and "An" are counted as one word
        String s = tokenizer.nextToken().toLowerCase().trim();
        word.set(s);
        context.write(word, one); // this writes the word and the number one, like this: ("Africa", 1)
      }
    }
  }

  public static class WordOccurrenceReducer
  extends Reducer<Text, IntWritable, Text, IntWritable>
  {
    private IntWritable occurrencesOfWord = new IntWritable();

    public void reduce(Text key,
                       Iterable<IntWritable> values,
                       Context context)
    throws IOException, InterruptedException
    {
      int sum = 0;
      for (IntWritable val : values) {
        sum += val.get();
      }
      occurrencesOfWord.set(sum);
      context.write(key, occurrencesOfWord); // this writes the word and the count, like this: ("Africa", 2)
    }
  }

  /**
   * the "driver" class. it sets everything up, then gets it started.
   */
  public static void main(String[] args)
  throws Exception
  {
    Configuration conf = new Configuration();
    String[] otherArgs = new GenericOptionsParser(conf, args).getRemainingArgs();
    if (otherArgs.length != 2)
    {
      System.err.println("Usage: wordcount <inputFile> <outputDir>");
      System.exit(2);
    }
    Job job = new Job(conf, "word count");
    job.setJarByClass(WordCount.class);
    job.setMapperClass(WordTokenizerMapper.class);
    job.setCombinerClass(WordOccurrenceReducer.class);
    job.setReducerClass(WordOccurrenceReducer.class);
    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);
    FileInputFormat.addInputPath(job, new Path(otherArgs[0]));
    FileOutputFormat.setOutputPath(job, new Path(otherArgs[1]));
    System.exit(job.waitForCompletion(true) ? 0 : 1);
  }
}
```

If Hadoop is installed, you can run this with
```bash
hadoop jar target/hadoop-examples-1.0.jar \
    com.thedataincubator.hadoopexamples.WordCount \
    gutenberg gutenberg-output
```

## General MapReduce Sequence

![MapReduce Process Structure](images/MapReduce_Work_Structure.png)
<!-- From Xiaocheng Zhang's blog: http://xiaochongzhang.me/blog/?p=338 -->

1. **Input**: the input is a large file on a distributed file system stored as an input system and read according to a user-specified `InputFormat`.  For word count, it is a large text file.
1. **Splitting**: an Input Reader splits the file into chunks of 64 MB or 128 MB.  We think of each chunk as consisting of a collection of records which can be represented as key-value pairs `(k0, v0)`.  We think of the records as being unordered but keyed.  The splitting is done so that each chunk is also on a single machine.  While there can be multiple chunks on a single machine, it is useful to abstract things and assume that each chunk resides on its own machine.
1. **Mapping**: the user specifies a `map` function, which iterates through the records within a chunk.  It outputs key-value pairs `(k1, v1)` (which are written to the local disk).  It can output multiple key-value pairs per input record or none at all and the key and value need not have the same type as the input.  In our case, each input record is a line in a file (e.g. `"Deer Bear River"`) and there is an output record consisting of each word with the number one (e.g. `("Deer", 1)`), indicating that we saw the word `"Deer"` once.
1. **Combiner**: This runs a reduce right after mapping but before shuffling.  The **local aggregation** can reduce the amount of I/O in the shuffle phase, which is the most important bottleneck.
1. **Shuffle**: up until this point, all the computations have happened locally and no data has been exchanged between the servers.  During Shuffle, the key value pairs `(k1, v1)` are sent to a computer based on `k1` so that all pairs with the same key end up at the same computer (and are written to that computer's disk).  Think of `k1` as giving the *address* to which to send the data and `v1` as giving the data payload to deliver.  Again, which we can have multiple keys mapping to the same physical machine, it is useful to abstract this and assume each key residing on its own machine.
1. **Reduce**: the user specifies a `reduce` function, which is given pairs `(k1, iter(v1))` where `iter(v1)` is an iterator over the values.  An iterator is like a list except that there is no random access to elements and one can only iterate through its elements once in order.  Underneath the hood, all the data resides on disk and only the current record being iterated over is kept in memory.  The `reduce` function outputs key-value pairs `(k2, v2)` (which are written to the local disk).  Again, the number of values it outputs is independent of the number of inputs and they can be of different types.  In our case, it maps each `(k1, iter(v1))` to `(k1, sum(v1))`.
1. **Final Result**: The results at this point are written out to disk according to a user-specified `OutputFormat`.

## Word Count in Spark

Spark is a distributed-computing frame work that can run on top of Hadoop or on its own.  It provides a somewhat more expressive way of describing distributed computations.  We will be covering it in much more detail in the rest of the notebooks, but here is a quick taste.

We start by importing modules and doing some set up.

In [ ]:
import os
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")

def localpath(path):
    return 'file://' + str(os.path.abspath(os.path.curdir)) + '/' + path

The following cell contains the entire word count process, which proceeds in much the same way as the MapReduce examples above.

In [ ]:
lines = sc.textFile(localpath('small_data/gutenberg/'))
lines.flatMap(lambda line: line.split(" ")) \
     .map(lambda word: (word.lower(), 1)) \
     .reduceByKey(lambda x, y: x + y) \
     .sortByKey() \
     .saveAsTextFile(localpath("gutenberg_count"))

In [ ]:
! cat gutenberg_count/part-00000 | grep "^('[a-z]" | head

The object `lines` is a **Resilient Distributed Dataset (RDD)**:

In [ ]:
type(lines)

This is like a (smartly) distributed list, which is partitioned across the nodes of the cluster and can be operated on in parallel.  The operations take the form of the usual functional list operations.  There are two types of operations: Transformations and Actions.

- **Transformations:** These creates a new RDD from an old one: for instance `map` or `flatMap`.  Transformations in Spark are __lazy__, which means that they do not compute the answer right away -- only when it is needed by something else.  Instead they represent the "idea" of the transformation applied to the base data set (e.g. for each chaining).  In particular, this means that intermediate results to computations do not necessarily get created and stored -- if the output of your map is going straight into a reduce, Spark should be clever enough to realize this and not actually store the output of the map.  Another consequence of this design is that the results of transformations are, by default, recomputed each time they are needed -- to store them one must explicitly call `cache`.
    
    [Transformations](https://spark.apache.org/docs/latest/programming-guide.html#transformations) typically *transform* the data and return another RDD.  Common examples include `map`, `filter`, `flatMap`, and also `join`, `cogroup`, `groupByKey`, `reduceByKey`, `countByKey`, and `sample`.

In [ ]:
lines.flatMap(lambda line: line.split(" "))

- **Actions:** These actually return a value as a result of a computation.  For instance, `reduce` is an action that combines all elements of an RDD using some function and then returns the final result.  (Note that `reduceByKey` actually returns an RDD.)
    
    [Actions](https://spark.apache.org/docs/latest/programming-guide.html#actions) typically either generate *small* outputs (e.g. `reduce`, `count`, `first`, `take`, `takeSample`, `foreach`, `collect`) or persist to disk (e.g. `saveAsTextFile`, `saveAsSequenceFile`, etc.).

In [ ]:
lines.flatMap(lambda line: line.split(" ")).count()

## Other Spark Features

### Spark DataFrames

While RDDs must be of consistent type, there is little structure on each row.  It is up to the user to remember the order of the data in each row.

Inspired by R, Spark **DataFrames** provide a structured view of data in an RDD, without compromising their distributed nature.

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
df = lines.flatMap(lambda line: line.split(" ")) \
          .map(lambda word: (word.lower(), 1)) \
          .reduceByKey(lambda x, y: x + y) \
          .toDF(['word', 'count'])

In [ ]:
df.printSchema()

In [ ]:
df.describe('count').show()

Transformations on the DataFrames are automatically carried out in parallel.

In [ ]:
df.filter(df['count'] > 5000).show()

### Spark SQL

We were initially using Spark for **batch processing**: We defined a pipeline through which all of the data travel, producing an answer.

DataFrames are part of Spark SQL, which allow you to write **queries** to run over distributed data.  You can do that with the syntax show above, or with SQL syntax.

In [ ]:
df.registerTempTable('counts')
sqlContext.sql('select * from counts where count > 5000').show()

### Machine Learning

Spark also supports **iterative processing**.  This includes machine learning algorithms like stochastic gradient descent.

Here, we use a linear model to work out the correlation between the length of words and their popularity.

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import UserDefinedFunction as udf
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.ml.linalg import VectorUDT, Vectors

In [ ]:
word_length = udf(lambda x: Vectors.dense(len(x)), VectorUDT())
feat_df = df.withColumn("features", word_length("word")) \
            .withColumnRenamed("count", "label")

In [ ]:
linreg = LinearRegression()
model = linreg.fit(feat_df)
model.coefficients

### Spark Streaming

Spark can also be used to perform **stream processing**.  Incoming data are grouped into "mini-batches", and the RDD transformations are applied to those batches.

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*